In [ ]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

In [ ]:
x_vec = np.array([
    1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980
], dtype=np.double)
y_vec = np.array([
    76_212_168,
    92_228_496,
    106_021_537,
    123_202_624,
    132_164_569,
    151_325_798,
    179_323_175,
    203_302_031,
    226_542_199,
], dtype=np.double)

Zadanie 1.
a) Najpierw wyznaczyliśmy współczynniki wielomianów dla `m=0..6` za pomocą wzoru $$A^T A c = A^T y$$ Wyznaczyliśmy macierz `A` funkcją `np.vander()`, a potem wektor `c` funkcją `np.linalg.solve()`. Następnie obliczyliśmy wartości wielomianu dla `x=1990` i wyznaczyliśmy błędy ekstrapolacji:

In [ ]:
c_vec = []
for m in range(7):
    A = np.vander(x_vec, m + 1, increasing = True)
    c = np.linalg.solve(A.T @ A, A.T @ y_vec)
    c_vec.append(c)

In [ ]:
correct_1990 = 248709873

def approx(x, m):
    return sum([c_vec[m][i] * x**i for i in range(m + 1)])

extrapolated_values = [approx(1990, m) for m in range(7)]

print("Wartości z ekstrapolacji:\n" + '\n'.join([f'{int(x)}, m={m}' for m, x in enumerate(extrapolated_values)]))
print("Błędy względne ekstrapolacji:\n" + '\n'.join([f'{abs(float((x - correct_1990) / correct_1990)):.4f}, m={m}' for m, x in enumerate(extrapolated_values)]))

Najmniejszy błąd względny otrzymaliśmy dla `m=6` równy `0.0032`. Dla wartości różnych od `m=0` błędy są małe co pokazuje skuteczność aproksymacji. 

b) Dla `m=0..6` wyznaczyliśmy kryterium informacyjne Akaikego ze wzoru $$\text{AIC} = 2k + n \ln \left( \frac{\sum_{i=1}^{n} |y_i - \hat{y}(x_i)|^2}{n} \right)$$ Ze względu na mały rozmiar próbki wyznaczyliśmy także kryterium ze składnikiem korygującym $$\text{AIC}_c = \text{AIC} + \frac{2k(k+1)}{n-k-1}$$ Otrzymaliśmy następujące wyniki:

In [ ]:
aic_c_vec = []
n = len(x_vec)
for m in range(7):
    k = m + 1
    s = sum([(y - approx(x, m))**2 for (x, y) in zip(x_vec, y_vec)])
    aic = 2 * k + n * np.log(s / n)
    aic_c = aic + (2 * k * (k + 1)) / (n - k - 1)
    aic_c_vec.append(aic_c)

print("Wartości AICc:\n" + '\n'.join([f'{float(x):.2f}, m={m}' for m, x in enumerate(aic_c_vec)]))

Mniejsze wartości kryterium oznaczają lepszy model, zatem według kryterium najlepszym modelem jest wielomian o stopniu `m=2`. Nie zgadza się to z poprzednio wyliczonymi błędami, które były mniejsze dla `m=4` i `m=6`.

Zadanie 2.
Najpierw zdefiniowaliśmy funkcje $$ T_0 = 1, \quad T_1 = x, \quad T_2 = 2x^2 - 1, \quad w = (1 - x^2)^{(-\frac{1}{2})} $$ Następnie wyznaczyliśmy współczynniki wielomiany ze wzoru $$ c_k = \frac{\langle f, T_k \rangle}{\langle T_k, T_k \rangle} $$ Przy całkowaniu uwzględniliśmy przesunięcie przedziału `[0, 2]` względem `[-1, 1]` $$ \langle f, T_k \rangle = \int_{0}^{2} w(x - 1)f(x)T_k(x - 1) \, dx $$ Potem wyznaczyliśmy wielomian aproksymacyjny ze wzoru $$ p_n = \sum_{k=0}^{n} c_k \phi_k $$

In [ ]:
f = lambda x: np.sqrt(x)
domain = (0, 2)
m = 2
T = lambda k, x: (1, x, 2 * x ** 2 - 1)[k]
w = lambda t: (1 - t ** 2) ** (-1 / 2)
TjTj = lambda j: np.pi if j == 0 else np.pi / 2

c_vec = [integrate.quad(lambda x: w(x - 1) * f(x) * T(j, x - 1), 0, 2)[0] / TjTj(j) for j in range(m + 1)]
p = lambda x: np.sum([c_vec[k] * T(k, x - 1) for k in range(m + 1)])
xs = np.linspace(0, 2, 100)

plt.figure(figsize=(12, 6))

plt.plot(xs, f(xs), label='f(x)')
plt.plot(xs, [p(x) for x in xs], label='Aproksymacja wielomianem 2 stopnia')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Wykres funkcji f(x) i jej aproksymacji')
plt.legend()
plt.grid(True)
plt.show()

Jak widać na wykresie wielomian aproksymacyjny 2 stopnia jest bardzo zbliżony do funkcji `f(x)` co pokazuje skuteczność aproksymacji.